In [103]:
import numpy as np
import tools, RNTK_avg
import jax
import symjax
import symjax.tensor as T

In [104]:
Ti = 2
TiPrime = 2

In [105]:
dataset = "trains"

In [106]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [107]:
N = int(dic["n_patrons1="])
length = int(dic["n_entradas="])

In [108]:
class RNTKTEST():
    def __init__(self):
        self.sw = 1
        self.su = 1
        self.sb = 1
        self.sh = 1
        self.L = 1
        self.Lf = 0
        self.sv = 1
        self.N = int(dic["n_patrons1="])
        self.length = int(dic["n_entradas="])
        
    def VT(self, M):
        A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
        # of all pairs of data in the data set
        B = A * A[:, None]
        C = T.sqrt(B)  # in R^{n*n}
        D = M / C  # this is lamblda in ReLU analyrucal formula
        E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
        F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
        G = (np.pi - T.arccos(E)) / (2 * np.pi)
        return F,G

In [109]:
rntk = RNTKTEST()

In [110]:
DATA = T.Placeholder((N, length), 'float32')
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [111]:
RNTK,GP = RNTK_avg.RNTK.RNTK_first(rntk,DATA[:,0])

In [112]:
l = 1

## LAYER ZERO ATTEMPT


In [113]:
switch_flag = 1
dim_1_i = dim_1 = 1
dim_2 = 1
dim_2_i = 0
dim_num = dim_1 + dim_2 + 1

lambdamatrix = np.zeros([1, dim_1 + 1, dim_2 + 1],dtype = object)
phimatrix = np.zeros([1, dim_1 + 1, dim_2 + 1],dtype = object)

print(dim_1_i, dim_2_i, switch_flag)
for d in range(0,dim_num):
    tiprime = dim_1_i
    ti = dim_2_i

    while ((tiprime <= dim_1) & (ti <= dim_2)):
        print("inner iteration - ", tiprime, ti) #// DIM 1 IS T PRIME, DIM 2 IS T
        if ((ti > 0) & (tiprime > 0)):
            S, D = rntk.VT(lambdamatrix[0, ti-1, tiprime-1])
            lambdamatrix[0, ti, tiprime] = rntk.sw ** 2 * S + rntk.su ** 2 * X + rntk.sb ** 2
            phimatrix[0, ti, tiprime] = lambdamatrix[0, ti, tiprime] + rntk.sw ** 2 * phimatrix[0, ti - 1, tiprime - 1] * D
        else:
            test = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + (rntk.su ** 2) * X + rntk.sb ** 2
            # phimatrix[0,ti,tiprime] = lambdamatrix[0,ti,tiprime] = T.expand_dims(test, axis = 0) # line 2, alg 1
            phimatrix[0,ti,tiprime] = lambdamatrix[0,ti,tiprime] = test # line 2, alg 1

        tiprime+=1
        ti+=1

    if dim_1_i == 0:
        switch_flag -= 1
    else:
        dim_1_i = dim_1_i - 1
    if switch_flag <= 0:
        dim_2_i = dim_2_i + 1

1 0 1
inner iteration -  1 0
inner iteration -  0 0
inner iteration -  1 1
inner iteration -  0 1


## MISC LOOKING AT OBJECT FORMATS

In [114]:
S_new,D_new = rntk.VT(gp_new[l-1])

In [115]:
S_new

Op(name=multiply_137, fn=multiply, shape=(10, 10), dtype=float32, scope=/default)

In [116]:
D_new

Op(name=true_divide_19, fn=true_divide, shape=(10, 10), dtype=float32, scope=/default)

In [117]:
extra = rntk.sh ** 2 * rntk.sw ** 2 * T.eye(n, n) + rntk.su**2 * S_new + rntk.sb**2

In [118]:
new = T.expand_dims(extra,axis = 0)

In [119]:
new

Op(name=expand_dims_27, fn=expand_dims, shape=(1, 10, 10), dtype=float32, scope=/default)

In [120]:
gp_new = T.concatenate([gp_new,new]) #line 4, alg 1

In [121]:
gp_new

Op(name=concatenate_13, fn=concatenate, shape=(3, 10, 10), dtype=float32, scope=/default)

## MIDDLE RNTK OBJECT TYPES

In [122]:
T.transpose(DATA[:, 1:])

Op(name=transpose_10, fn=transpose, shape=(28, 10), dtype=float32, scope=/default)

In [123]:
T.stack([RNTK,GP])

Op(name=stack_10, fn=stack, shape=(2, 2, 10, 10), dtype=float32, scope=/default)

In [124]:
r1, r2 = T.scan(lambda a,b:middletest(a,b),sequences=[ T.transpose(DATA[:, 1:]) ], init=T.stack([RNTK,GP]))

a[0] Op(name=_rewriting_take_56, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
a[1] Op(name=_rewriting_take_57, fn=_rewriting_take, shape=(2, 10, 10), dtype=float32, scope=/default)
B Placeholder(name=_19, shape=(10,), dtype=float32, scope=/default)


In [125]:
def middletest(a,b):
    # print("A", a)
    print("a[0]", a[0])
    print("a[1]", a[1])
    print("B", b)
    return a,b